In [29]:
import sys, os
sys.path.append(os.path.join(os.path.dirname('__file__'), '..', 'DB_and_Azure'))
import sql_db_functions as SQLf

import pandas as pd
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
 

import getpass
from apikey import apikey 


In [30]:
def get_products():

    conn, cursor = SQLf.sql_db_functions.connect_sql()

    query = f"SELECT Brand_Prod_ID, Descript  FROM Products ;" #LIMIT 5
    cursor.execute(query)

    # Fetch the rows
    rows = cursor.fetchall()

    df = pd.DataFrame(data=rows, columns=['Brand_Prod_ID','Descript'])

    SQLf.sql_db_functions.close_connection_db(conn=conn, cursor=cursor)

    return df


In [31]:
def get_related_links(Brand_Prod_ID):
    conn, cursor = SQLf.sql_db_functions.connect_sql()

    query = f"SELECT image_link FROM product_img WHERE Brand_id = {Brand_Prod_ID} ;"
    cursor.execute(query)
    # Fetch the rows
    image_links = cursor.fetchall()

    SQLf.sql_db_functions.close_connection_db(conn=conn, cursor=cursor)

    df_images = pd.DataFrame(data=image_links, columns=['image_link'])
    df_images.head()

    df_images['image_link'] = df_images['image_link'].apply(lambda x: x if 'https' in x else 'https:' +  x )

    return df_images.image_link

In [32]:
conn, cursor = SQLf.sql_db_functions.connect_sql()

query = f"SELECT image_link FROM product_img ;"
cursor.execute(query)
# Fetch the rows
image_links = cursor.fetchall()

SQLf.sql_db_functions.close_connection_db(conn=conn, cursor=cursor)

df_images = pd.DataFrame(data=image_links, columns=['image_link'])
df_images.head()

df_images['image_link'] = df_images['image_link'].apply(lambda x: x if 'https' in x else 'https:' +  x )

In [33]:
def generate_prompt(image_urls, description,prompt_instructions):
    # Construct the image part of the prompt
    images_text = ' '.join([f"{url}" for url in image_urls])
    # Combine with the description
    prompt_text = f"""
        IMAGES: {images_text} \n
        DESCRIPTION: {description} \n
        
        {prompt_instructions}
    """
    return prompt_text

In [34]:
def get_gpt_description(current_images, current_description):

    prompt_instructions = """

    The images showcase the same Top - Shirt, use them to create a detailed description of it and its style.
    \n
    You MUST identify each of these characteristics in the IMAGES or DESCRIPTION to create the detailed description characteristics: 
    \n
    Fit, Material, Formality, Seasson, Colors, Texture, Transparency, Details and Embellishments, Shape,
    Length, Fluidity of fabric, Fabric weight, Breathability, Occasion Suitability, etc.

    Try to identify and describe the following chategories in the IMAGES or DESCRIPTION : 
    \n
    Sleeve style, Neckline, Collar Style, Patterns, Patterns placement, Pocket Presence, Pocket placement, Pocket size, and Lapel 
    \n
    \n

    DONT DESCRIBE THE MODEL.
    Use information from the IMAGES and the DESCRIPTION.
    IGNORE BACKGROUND OF THE IMAGES.  
    ANSWER MUST BE AS SPECIFIC AS POSSIBLE BUT NOT COMPLEX
    WRITE IN A SINGLE PARAGRAPH
    NAME AND DESCRIBE THE COLORS USED
    """

    prompt_text = generate_prompt(current_images, current_description,prompt_instructions)


    os.environ['OPENAI_API_KEY'] = apikey

    # LLM
    model = ChatOpenAI(model="gpt-4o", temperature=0.6)

    # Prompt template
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "You are a fashion specialist."),
            ("user", prompt_text),
        ]
    )

    chain = prompt | model

    response = chain.invoke({})
    return response.content



In [35]:
def get_gpt_form(current_images, current_description):

    prompt_instructions = """

    The IMAGES show the same Top - Shirt. Identify the product in the IMAGES and use the IMAGES and DESCRIPTION to fill out the following format to list attributes of the clothing piece in the image, separated by commas.  
    \n
    \n
    *Type:  
    *Fit: (NotSlim fit, regular fit, oversized, tight, loose, etc)
    *Sleeve style: (Not Applicable, Short sleeve, long sleeve, sleeveless, cap sleeve, etc)     
    *Neckline: (Not Applicable, Crew neck, V-neck, scoop neck, etc)   
    *Material: (Cotton, linen, silk, polyester, etc)
    *Formality: (Casual, Business Casual, Formal, etc)
    *Seasson: (Winter, Spring, Summer, Autum)
    *Colors:  (percentage of each color in the clothing peace) 
    *Texture: (Smooth, Rough, Ribbed, Velvety, etc)
    *Transparency: (Opaque, Semi-transparent, Transparent, etc)
    *Details and Embellishments: (e.g, buttons, zippers, embroidery, etc)
    *Shape: (e.g., boxy, fitted, flared, etc) 
    *Length: (e.g., cropped, hip-length, tunic, etc)
    *Collar Style: (e.g., Not Applicable, crew neck, V-neck, polo, button-down)
    *Patterns: (e.g., stripes, floral, geometric, not applicable)
    *Patterns placement: (e.g., lower back, left sholder, right chest, not applicable) 
    *Fluidity of fabric:
    *Fabric weight: (Light, medium, or heavy)
    *Pocket Presence:(Yes,No)
    *Pocket placement:(e.g., lower back, left sholder, right chest, not applicable)
    *Pocket size:(small, medium, big, not applicable)
    *Breathability: (Low, Medium, High)
    *Occasion Suitability: (Casual, formal, business casual, etc)
    *Lapel: (not applicable, Notch, Peak, Shawl Satin, Slim, Wide, Contrasting, etc)

    \n
    \n
    Use information from the images and the description.
    IGNORE BACKGROUND.
    ONLY ANSWER THE FORMAT.
    IF THERE IS A MODEL IGNORE IT. 
    DONT USE THE BRAND NAME IN ANY DESCRIPTION. 
    YOU CAN USE MULTIPLE DESCRIPTIONS IN A SINGLE ATTRIBUTE.  
    DONT USE MULTICOLOR OR Multicolor, NAME THE SPECIFIC COLORS.  
    COLORS SHOULD INCLUDE PRINT AND CLOTHE COLORS WITHOUT ANY DESCRIPTION OF THE PRINT ONLY MAIN COLORS.
   
    """

    prompt_text = generate_prompt(current_images, current_description,prompt_instructions)


    os.environ['OPENAI_API_KEY'] = apikey

    # LLM
    model = ChatOpenAI(model="gpt-4o", temperature=0.6)

    # Prompt template
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "You are a fashion specialist."),
            ("user", prompt_text),
        ]
    )

    chain = prompt | model

    response = chain.invoke({})
    return response.content



In [36]:
def load_form_to_db(current_product_ID,form, Summary):

    
    conn, cursor = SQLf.sql_db_functions.connect_sql()

    query = """
    INSERT INTO product_characteristics (Brand_id, Detail, Summary) VALUES (%s, %s, %s)
    RETURNING Brand_id;
    """
    #INSERT INTO product_characteristics (Brand_id, Detail, Summary) VALUES (%s, %s, %s)

    cursor.execute(query, (current_product_ID, form, Summary))

    # Fetch the rows
    #rows = cursor.fetchall()

    conn.commit()

    SQLf.sql_db_functions.close_connection_db(conn=conn,cursor=cursor)

    #return rows

    


In [37]:
def update_form_in_db(current_product_ID, form, Summary):
    
    conn, cursor = SQLf.sql_db_functions.connect_sql()

    query = """
    UPDATE product_characteristics 
    SET Detail = %s, Summary = %s
    WHERE Brand_id = %s;
    """

    cursor.execute(query, (form, Summary, current_product_ID))

    conn.commit()

    SQLf.sql_db_functions.close_connection_db(conn=conn,cursor=cursor)

In [38]:
df = get_products()

In [39]:
df.shape

(154, 2)

In [40]:
df.Brand_Prod_ID.unique().shape

(154,)

In [41]:
df_final= pd.DataFrame(columns=['Brand_Prod_ID','current_image_links','cloth_summary','cloth_form'])
n_rows = len(df)

for current_product_ID in df.Brand_Prod_ID:

    current_image_links = get_related_links(current_product_ID)
    current_description = df[df.Brand_Prod_ID == current_product_ID]['Descript']

    cloth_summary = get_gpt_description(
        current_images=current_image_links,
        current_description=current_description
        )
    
    
    cloth_form = get_gpt_form(
        current_images=current_image_links,
        current_description=cloth_summary + """/n     /n""" + current_description
        )
    
    temp_df = pd.DataFrame([{
        'Brand_Prod_ID':current_product_ID,
        'current_image_links':current_image_links,
        'cloth_summary':cloth_summary,
        'cloth_form':cloth_form
        }])
    
    df_final = pd.concat(
            [df_final,temp_df],
            ignore_index=True
        )
    
    #load_form_to_db(current_product_ID = current_product_ID,form=cloth_form, Summary= cloth_summary)
    update_form_in_db(current_product_ID = current_product_ID,form=cloth_form, Summary= cloth_summary)
    
    if current_product_ID % 10 == 0:
        print(f"loaded {(current_product_ID)/n_rows}")

    elif current_product_ID == 0:
        print(f"loaded {current_product_ID}")

loaded 0.3246753246753247
loaded 0.06493506493506493
loaded 0.12987012987012986
loaded 0.19480519480519481
loaded 0.2597402597402597
loaded 0.38961038961038963
loaded 0.45454545454545453
loaded 0.5194805194805194
loaded 0.5844155844155844
loaded 0.6493506493506493
loaded 0.7142857142857143
loaded 0.7792207792207793
loaded 0.8441558441558441
loaded 0.9090909090909091
loaded 0.974025974025974


# checking !!!

In [14]:
Brand_Prod_ID = 25

conn, cursor = SQLf.sql_db_functions.connect_sql()

query = f"SELECT Brand_Prod_ID, Descript  FROM Products WHERE Brand_Prod_ID = {Brand_Prod_ID}  ;" #LIMIT 5
cursor.execute(query)

# Fetch the rows
rows = cursor.fetchall()

df = pd.DataFrame(data=rows, columns=['Brand_Prod_ID','Descript'])

SQLf.sql_db_functions.close_connection_db(conn=conn, cursor=cursor)


conn, cursor = SQLf.sql_db_functions.connect_sql()

query = f"SELECT image_link FROM product_img WHERE Brand_id = {Brand_Prod_ID} ;"
cursor.execute(query)
# Fetch the rows
image_links = cursor.fetchall()

SQLf.sql_db_functions.close_connection_db(conn=conn, cursor=cursor)

df_images_temp = pd.DataFrame(data=image_links, columns=['image_link'])
df_images_temp.head()

df_images_temp['image_link'] = df_images_temp['image_link'].apply(lambda x: x if 'https' in x else 'https:' +  x )


In [15]:
df.head()

,Brand_Prod_ID,Descript
0,20,Ispirato allo spirito estivo e ai beach club d...


In [16]:
df.Descript[0]

"Ispirato allo spirito estivo e ai beach club della costa italiana, questo articolo fa parte della linea Gucci Lido. I codici emblematici della Maison e gli eleganti motivi vengono reinterpretati in chiave contemporanea per la collezione Pre-Fall. Questa maglia dal taglio dritto in twill di seta è definita da una stampa Morsetto Gucci e righe, che rende omaggio alla tradizione equestre della Maison.  Twill di seta marrone e verde con stampa Morsetto Gucci e righe Profilo bianco Senza fodera Girocollo Maniche corte Chiusura con bottoni oversize Lunghezza: 77 cm; taglia 42 (IT) Fabbricato in Italia Il prodotto raffigurato nell'immagine è taglia 42 (IT)"

In [17]:
for i in df_images_temp.image_link:
    print(i)

https://media.gucci.com/style/DarkGray_Center_0_0_980x980/1711564261/785724_ZAQTF_2054_001_100_0000_Light-Maglia-in-seta-con-stampa-Morsetto-Gucci-e-righe.jpg
https://media.gucci.com/style/DarkGray_Center_0_0_980x980/1711564264/785724_ZAQTF_2054_004_100_0000_Light-Maglia-in-seta-con-stampa-Morsetto-Gucci-e-righe.jpg
https://media.gucci.com/style/DarkGray_Center_0_0_980x980/1711564264/785724_ZAQTF_2054_005_100_0000_Light-Maglia-in-seta-con-stampa-Morsetto-Gucci-e-righe.jpg
https://media.gucci.com/style/White_Center_0_0_730x490/1711564261/785724_ZAQTF_2054_001_100_0000_Light-Maglia-in-seta-con-stampa-Morsetto-Gucci-e-righe.jpg


## check only prod

In [18]:
current_product_ID = Brand_Prod_ID

current_image_links = get_related_links(current_product_ID)
current_description = df[df.Brand_Prod_ID == current_product_ID]['Descript']

In [20]:
current_description[0]

"Ispirato allo spirito estivo e ai beach club della costa italiana, questo articolo fa parte della linea Gucci Lido. I codici emblematici della Maison e gli eleganti motivi vengono reinterpretati in chiave contemporanea per la collezione Pre-Fall. Questa maglia dal taglio dritto in twill di seta è definita da una stampa Morsetto Gucci e righe, che rende omaggio alla tradizione equestre della Maison.  Twill di seta marrone e verde con stampa Morsetto Gucci e righe Profilo bianco Senza fodera Girocollo Maniche corte Chiusura con bottoni oversize Lunghezza: 77 cm; taglia 42 (IT) Fabbricato in Italia Il prodotto raffigurato nell'immagine è taglia 42 (IT)"

In [26]:
current_description.values[0]

"Ispirato allo spirito estivo e ai beach club della costa italiana, questo articolo fa parte della linea Gucci Lido. I codici emblematici della Maison e gli eleganti motivi vengono reinterpretati in chiave contemporanea per la collezione Pre-Fall. Questa maglia dal taglio dritto in twill di seta è definita da una stampa Morsetto Gucci e righe, che rende omaggio alla tradizione equestre della Maison.  Twill di seta marrone e verde con stampa Morsetto Gucci e righe Profilo bianco Senza fodera Girocollo Maniche corte Chiusura con bottoni oversize Lunghezza: 77 cm; taglia 42 (IT) Fabbricato in Italia Il prodotto raffigurato nell'immagine è taglia 42 (IT)"

In [21]:
cloth_summary = get_gpt_description(
    current_images=current_image_links,
    current_description=current_description
)

In [27]:
cloth_summary

"The showcased top is a Gucci silk shirt inspired by the summer spirit and beach club aesthetics. It features a relaxed, loose fit that ensures comfort and ease of movement, suitable for warm weather. Constructed from lightweight, breathable silk, the shirt is ideal for summer and beach vacations. The primary color of the shirt is a rich, earthy brown, accented with cream and dark green stripes that run vertically down the shirt, enhancing its elongated shape. The shirt's texture is smooth and luxurious, with a slight sheen typical of high-quality silk. The fabric is lightweight and fluid, which adds to its breezy and airy feel. The shirt has long sleeves with buttoned cuffs, a classic point collar, and a button-down front, contributing to its semi-formal to casual versatility.\n\nThe shirt is adorned with Gucci's signature Morsetto (horsebit) print in a contrasting dark green, placed strategically between the vertical stripes, giving it a distinctive and recognizable look. There are n

In [23]:
cloth_form = get_gpt_form(
        current_images=current_image_links,
        current_description=cloth_summary + """/n     /n""" + current_description
        )

In [28]:
cloth_form

'*Type: Top - Shirt  \n*Fit: Regular fit  \n*Sleeve style: Long sleeve  \n*Neckline: Not applicable  \n*Material: Silk  \n*Formality: Formal  \n*Season: Spring, Summer, Autumn  \n*Colors: Beige (70%), Green (15%), Red (10%), Blue (5%)  \n*Texture: Smooth  \n*Transparency: Opaque  \n*Details and Embellishments: Buttons  \n*Shape: Fitted  \n*Length: Hip-length  \n*Collar Style: Button-down  \n*Patterns: Stripes, Geometric  \n*Patterns placement: All over  \n*Fluidity of fabric: High  \n*Fabric weight: Light  \n*Pocket Presence: No  \n*Pocket placement: Not applicable  \n*Pocket size: Not applicable  \n*Breathability: High  \n*Occasion Suitability: Formal, Business casual  \n*Lapel: Not applicable'